In [8]:
import pandas as pd
import numpy as np
import warnings
from gensim.models import Word2Vec
warnings.filterwarnings('ignore')

In [9]:
movies= pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

In [10]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [11]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [12]:
# merging the both df on title for convience 
movies = movies.merge(credits, on='title')

In [13]:
movies.shape

(4809, 23)

In [14]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [15]:
# idetifying the features which will add value in our recommendation 

# genres
# id --> to fetch the poster of movie from site when we have to deploy the app
# keywords
# title
# overview
# cast
# crew

new_mov = movies[['movie_id', 'title', 'overview', 'genres','keywords','cast', 'crew']]

In [16]:
#discarding original language because of imbalanced data(most of the movies are in english so considering it will mess up things)
#movies.original_language.value_counts()


# discarding original language bcuz the title can be also in hindi.
# also other cols are numeric so discarding those

In [17]:
new_mov.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [18]:
new_mov.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [19]:
# dropping the nulls as there are only 3 values.
new_mov.dropna(inplace=True)

In [20]:
# to check duplicate value
new_mov.duplicated().sum()

0

In [21]:
# now we have to preprocess the data of genres cast, crew, keywords to make a new col Tags which is combination of all those cols

In [22]:
new_mov['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [8]:
import ast

def convert(obj):
    L=[]
    # now here genres are list inside string('[]') so we have to first convert them to list for that we will use ast function. 
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [4]:
import ast

In [9]:
obj = '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
r = convert(obj)

In [10]:
r

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [1]:
eval('{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}')

({'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'})

In [24]:
ast.literal_eval('{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}')

({'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'})

In [20]:
new_mov['genres'] = new_mov['genres'].apply(convert)

In [25]:
new_mov.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [26]:
new_mov.keywords[0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [27]:
# we will apply the same function created to extract values from keywords column
new_mov.keywords = new_mov.keywords.apply(convert)

In [28]:
new_mov.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [25]:
# now for cast col we will extract only 3 dict from that as we need only those top 3 characters name from it

In [30]:
# modifying above function to extract only 3 values
poshhdhhn

In [31]:
new_mov['cast'] = new_mov['cast'].apply(convert3)

In [32]:
new_mov.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [33]:
def fetch(obj):
    L=[]
    # now here crew are list inside string so we have to first convert them to list for that we will use ast function. 
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [34]:
new_mov.crew = new_mov.crew.apply(fetch)

In [35]:
new_mov.overview[0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [36]:
new_mov.overview = new_mov.overview.apply(lambda x: x.split())

In [37]:
new_mov.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]


##### now there is one prblm with our data if u  see in the cast col there is name 'Sam Worthington' so now if we feed data like this then our system will consider this as two different names so we don't want that so we will be removing the extra space from the words 
e.g: there are two sam's in the list if wanted to see  Worthingtons movie but as we didn't pre processed the data the system will get confused between the two sam (Sam Mendes, Sam Worthington) so to avoid this prblm we will remove the space so now system will consider SamMendes as one entity and SamWorthington as diff.


In [40]:
new_mov.genres = new_mov.genres.apply(lambda x : [i.replace(" ", "") for i in x])

In [41]:


new_mov.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[[, {, "", i, d, "", :, , 2, 8, ,, , "", n, a, m,...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[[, {, "", i, d, "", :, , 1, 2, ,, , "", n, a, m,...","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [42]:
new_mov.keywords = new_mov.keywords.apply(lambda x : [i.replace(" ", "") for i in x])
new_mov.cast = new_mov.cast.apply(lambda x : [i.replace(" ", "") for i in x])
new_mov.crew = new_mov.crew.apply(lambda x : [i.replace(" ", "") for i in x])

In [37]:
new_mov.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]


In [43]:
# creating tags from the 5 cols which we preprocessed
new_mov['tags'] = new_mov['overview'] + new_mov['genres'] + new_mov['keywords'] +new_mov['cast']+ new_mov['crew']

In [44]:
new_mov.head(3)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[[, {, "", i, d, "", :, , 2, 8, ,, , "", n, a, m,...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[[, {, "", i, d, "", :, , 1, 2, ,, , "", n, a, m,...","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[[, {, "", i, d, "", :, , 2, 8, ,, , "", n, a, m,...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."


In [40]:
df = new_mov[['movie_id', 'title', 'tags']]

In [41]:
df.head(3)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."


In [42]:
df['tags'] = df['tags'].apply(lambda x: " ".join(x))

In [43]:
df.head(3)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...


In [44]:
df['tags'] = df['tags'].apply(lambda x: x.lower())

In [45]:
df['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [46]:
# df.to_csv("transformed.csv")

In [47]:
import pickle
import pandas as pd

In [48]:
similarity = pickle.load(open("similarity.pkl", 'rb'))
mov_list = pickle.load(open('movlist.pkl','rb'))
movdict=pickle.load(open('movdict.pkl','rb'))
movies= pd.DataFrame(movdict)

In [49]:
df = pd.read_csv("transformed.csv")

In [50]:
import nltk

In [51]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [52]:
ps.stem('dance')

'danc'

In [53]:
def stem(text):
    l=[]
    for i in text.split():
       l.append(ps.stem(i))
    return " ".join(l)

In [54]:
df['tags'] = df['tags'].apply(stem)

In [55]:
df.head(3)

,Unnamed: 0,movie_id,title,tags
0,0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,2,206647,Spectre,a cryptic messag from bond’ past send him on a...


In [56]:
from nltk.corpus import stopwords

In [57]:
def remove_stopwords(text):
    l = []
    for i in text.split():
        if i not in set(stopwords.words('english')):
            l.append(i)
    return " ".join(l)

In [60]:
df['tags'] = df['tags'].apply(remove_stopwords)

In [120]:
df['tags'][0]

'22nd century, parapleg marin dispatch moon pandora uniqu mission, becom torn follow order protect alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [65]:
model1 = Word2Vec(df['tags'], min_count = 1,vector_size = 5000, window = 5, sg=0)

In [113]:
word2vec = model1.wv.vectors_for_all

In [ ]:
model1.wv.most_similar('captain',topn=5)

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [76]:
vec = cv.fit_transform(df['tags']).toarray()

In [114]:
len(word2vec)

TypeError: object of type 'method' has no len()

In [112]:
len(vec[0])

5000

In [10]:
cv.get_feature_names()

F:\anaconda\envs\Recommendation_System\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '18th',
 '19',
 '1930s',
 '1940s',
 '1950',
 '1950s',
 '1960s',
 '1970s',
 '1980',
 '1980s',
 '1985',
 '1990s',
 '1999',
 '19th',
 '19thcentury',
 '20',
 '200',
 '2009',
 '20th',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '60s',
 '70',
 '70s',
 'aaron',
 'aaroneckhart',
 'abandoned',
 'abducted',
 'abigailbreslin',
 'abilities',
 'ability',
 'able',
 'aboard',
 'abuse',
 'abusive',
 'academy',
 'accept',
 'accepted',
 'accepts',
 'access',
 'accident',
 'accidental',
 'accidentally',
 'accompanied',
 'accomplish',
 'account',
 'accountant',
 'accused',
 'ace',
 'achieve',
 'act',
 'acting',
 'action',
 'actionhero',
 'actions',
 'activist',
 'activities',
 'activity',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'adam',
 'adams',
 'adamsandler',
 'adamshankman',
 'adaptation',
 'adapted',
 'addict',
 'addicted',
 'addiction',
 'adolescence',
 'adolescent'

In [109]:
# so now we have to calculate the distance between two vectors, we will not use eucledian distance as it fails in higher
#dimension(we are oin 5000D).In higher Dimension eucledian distance is not a reliable dist.
# instead we will use cosine similarity to calculate the similarity between two vectors. Distance is inversly prop to Similarity

In [80]:
from sklearn.metrics.pairwise import cosine_similarity

In [84]:
similarity = cosine_similarity(vec)

In [115]:
similarity1 = cosine_similarity(word2vec)

TypeError: float() argument must be a string or a number, not 'method'

In [88]:
len(similarity)

4806

In [106]:
len(similarity1)

152

In [14]:
sorted(list(enumerate(similarity[1])), reverse=True, key=lambda x: x[1])[1:6]

[(12, 0.4140043409440133),
 (199, 0.27500954910846337),
 (17, 0.2533201985524494),
 (216, 0.20579830217101058),
 (3572, 0.20579830217101058)]

In [15]:
def recommend(movie):
    mov_index = df[df['title'] == movie].index[0]
    dist = similarity[mov_index]
#here we are retaining the index of movies by enumerating and indexing from 1:6 as 0th position will be the similarity with itself.
    mov_list = sorted(list(enumerate(dist)), reverse=True, key=lambda x: x[1])[1:6]
    for i in mov_list: 
        print(df.iloc[i[0]].movie_id)
        print(df.iloc[i[0]].title)


In [84]:
new_mov[new_mov['title'] == 'Avatar'].index

Int64Index([0], dtype='int64')

In [16]:
recommend('Avatar')

440
Aliens vs Predator: Requiem
679
Aliens
270938
Falcon Rising
602
Independence Day
7450
Titan A.E.


In [87]:
import pickle

In [88]:
pickle.dump(df.to_dict(), open('movdict.pkl', 'wb'))

In [89]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))